# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [16]:
# %% [markdown]
# # Export Excel Spells to FoundryVTT Compendium JSON
#
# This notebook reads the "Spells" sheet from the Excel file and converts each row into a JSON object formatted for FoundryVTT.
# You may need to adjust the column names if they differ from your Excel file.

# %%
import pandas as pd
import json
from PIL import Image, ImageDraw, ImageFont
import inflection

# Path to the Excel file
file_path = "data/Vestigium_Spells.xlsx"

# Load the "Spells" sheet into a DataFrame
df_spells = pd.read_excel(file_path, sheet_name="Spells")

# Display the first few rows to verify data
df_spells.head()

# %% [markdown]
# ## Define a Function to Convert a Row to a FoundryVTT Spell
#
# This function takes a row from the DataFrame and maps its columns to the JSON structure required by FoundryVTT.
#
# **Note:** Adjust the field names and default values as needed.
# For example, if your Excel uses different column headers (e.g., "CastingTime" instead of "Casting Time"), update accordingly.

def generate_spell_icon(spell_name, image_size=(128, 128), background_color="white", text_color="black", font_size=24):
  # Create a new image with a solid background
  img = Image.new("RGB", image_size, background_color)
  draw = ImageDraw.Draw(img)

  # Attempt to load a truetype font; fallback to default if not found
  try:
    font = ImageFont.truetype("arial.ttf", font_size)
  except IOError:
    font = ImageFont.load_default()

  # Calculate text width and height to center the text
  _, _, width, height = draw.textbbox((0, 0), text=spell_name, font=font)
  text_x = (image_size[0] - width) / 2
  text_y = (image_size[1] - height) / 2

  # Draw the spell name on the image
  draw.text((text_x, text_y), spell_name, fill=text_color, font=font)
  file_name = f"images/{inflection.underscore(spell_name.replace("/", "_"))}.png"
  img.save(file_name)
  return file_name
# %%
def row_to_spell(row):
  # Basic normalization
  components_str = row.get("Components", "")
  components_set = {comp.strip().upper() for comp in str(components_str).split(",")}
  spell_classes = [cls.strip() for cls in str(row.get("Class", "")).split(",") if cls.strip()]
  ability_checks = [cls.strip() for cls in str(row.get("Ability Check", "")).split(",") if cls.strip()]
  misc_tags = [cls.strip() for cls in str(row.get("Foundry Tag", "")).split(",") if cls.strip()]
  damages = [cls.strip() for cls in str(row.get("Damage Type", "")).split(",") if cls.strip()]
  saving_throws = [cls.strip() for cls in str(row.get("Saving Throw", "")).split(",") if cls.strip()]
  areas = [cls.strip() for cls in str(row.get("Area ABRV", "")).split(",") if cls.strip()]
  # Parse components if they are stored as a string like "V, S, M"
  components_str = row.get("Components ABVR", "")
  duration_type = row.get("Duration Type") if not pd.isnull(row.get("Duration Type")) else "timed"
  duration_unit = row.get("Duration Unit") if not pd.isnull(row.get("Duration Unit")) else "minutes"
  duration_amount = row.get("Duration Amount") if not pd.isnull(row.get("Duration Amount")) else 1
  range_distance = row.get("Range Distance") if not pd.isnull(row.get("Range Distance")) else "self"
  
  # Normalize the components string (if needed)
  components_set = {comp.strip().upper() for comp in str(components_str).split(",")}
  # Range formatting
  img_path = generate_spell_icon(row.get("Spell Name", "Unnamed Spell"))
  # Construct the spell
  spell = {
    "name": row.get("Spell Name", "Unnamed Spell"),
    "level": int(row["Level"][0]) if not pd.isnull(row.get("Level")[0]) else 0,
    "school": row.get("School ABRV", "E"),
    "time": [
      {
        "number": row.get("Casting Unit", 1),
        "unit": row.get("Casting Type", "Action"),
      }
    ],
    "time": {
      "type": row.get("Range Type", "Point"),
      "distance": {
        "type": range_distance,
        **({"amount": row.get("Range Unit")} if not pd.isnull(row.get("Range Unit")) else {})
      }
    },
    "duration": [
      {
        "type": duration_type,
        **({"duration": {
          "type": duration_unit,
          "amount": duration_amount,
          "upTo": True if row.get("Up To", "FALSE") == "TRUE" else False
        }} if duration_type == "timed" else {}),
        **({"concentration": True if row.get("Concentration", "FALSE") == "TRUE" else False} if duration_type == "timed" else {}),
      }
    ],
    "classes": {
      "fromClassList": [
        {
          "name": cls,
          "source": "VSTG"
        } for cls in spell_classes
      ]
    },
    "entries": [
      row.get("Description")
    ] + ([row.get("Clarification")] if not pd.isnull(row.get("Clarification")) else []) 
      + ([row.get("Table")] if not pd.isnull(row.get("Table")) else []),
    "source": "Vestigium",
    **({"entriesHigherLevel": [
        {
        "type": "entries",
        "name": "At Higher Levels",
        "entries": [row.get("Higher Levels", "")] if not pd.isnull(row.get("Higher Levels")) else []
        }
      ]} if not pd.isnull(row.get("Higher Levels")) else {}),
    "components": {
      "v": "V" in components_set,
      "s": "S" in components_set,
      "r": "R" in components_set,
      "t": "T" in components_set
    },
    **({"abilityCheck": ability_checks} if not pd.isnull(row.get("Ability Check")) else {}),
    **({"miscTags": misc_tags} if not pd.isnull(row.get("Foundry Tag")) else {}),
    **({"damageInflict": damages} if not pd.isnull(row.get("Damage Type")) else {}),
    "fluff": {
      "entries": [
      ] + ([row.get("Flavor")] if not pd.isnull(row.get("Flavor")) else [])
        + ([row.get("Alternative Flavor")] if not pd.isnull(row.get("Alternative Flavor")) else [])
        + ([row.get("Quotes")] if not pd.isnull(row.get("Quotes")) else []),
      "images": [
        {
          "type": "image",
          "href": {
            "type": "external",
            "url": generate_spell_icon(row.get("Spell Name", "Unamed Spell"),)
          }
        }
      ]
    },
    **({"savingThrow": saving_throws } if not pd.isnull(row.get("Saving Throw")) else {}),
    **({"areaTags": areas} if not pd.isnull(row.get("Area ABRV")) else {}),
  }
  return spell

# %% [markdown]
# ## Convert All Spells and Export to JSON
#
# The following cell loops through each row in the DataFrame, converts it to the required JSON format,
# and then writes all spells into a JSON file wrapped in a compendium container.

# %%
spells_list = [
  row_to_spell(row)
  for index, row in df_spells.iterrows()
  if pd.notnull(row.get("Spell Name")) and str(row.get("Spell Name")).strip() != ""
]
# Wrap the spells in a compendium container (adjust the "pack" id as needed)
compendium = {
  "pack": "compendium.spells",
  "documents": spells_list
}

# Write the compendium to a JSON file
output_file = "spells_compendium.json"
with open(output_file, "w", encoding="utf-8") as f:
  json.dump(compendium, f, ensure_ascii=False, indent=4)

print(f"Export successful! The compendium has been saved as '{output_file}'.")

Export successful! The compendium has been saved as 'spells_compendium.json'.
